#**Requirement 2: Nearest Neighbor Search and Duplicate Detection**

In [116]:
import pandas as pd
import numpy as np

## **Question​ 2b: Same Question Detection**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [129]:
train = pd.read_csv('drive/My Drive/Fall-2020/Big Data/Question2/train.csv')
train

,Id,Question1,Question2,IsDuplicate
0,0,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0
...,...,...,...,...
282999,282999,What is the highest gallantry award in India?,India's PM Mr Modi has polled highest votes fr...,0
283000,283000,Who are your favorite Quorans?,Who are some of your favorite Quorans and why?,1
283001,283001,What is the best way to add your websites back...,What is the best way to add your websites back...,0
283002,283002,How old is too old to start medical school?,What age is too old to get a medical MD?,1


In [161]:
test = pd.read_csv('drive/My Drive/Fall-2020/Big Data/Question2/test_without_labels.csv')
test

,Id,Question1,Question2
0,283003,What can someone do if they've lost the wirele...,What is the best USB wireless mouse that can b...
1,283004,Why India need to elect Prime minister?,Is prime minister of India elected or appointed?
2,283005,How can I make money online with free of cost?,How can I make money online for free?
3,283006,Does MDMA affect the first and higher order mo...,Do antipsychotics affect the first and higher ...
4,283007,"I am a Saudi National and have ""SR 3 million"" ...",Where should I invest money to get high returns?
...,...,...,...
121282,404285,How many keywords are there in the Racket prog...,How many keywords are there in PERL Programmin...
121283,404286,Do you believe there is life after death?,Is it true that there is life after death?
121284,404287,What is one coin?,What's this coin?
121285,404288,What is the approx annual cost of living while...,I am having little hairfall problem but I want...


In [162]:
train.isnull().values.any()
train = train.replace(np.nan, '', regex=True)
test = test.replace(np.nan, '', regex=True)

In [132]:
train.isnull().values.any()

False

In [163]:
test.isnull().values.any()

False

In [ ]:
train.duplicated().unique()

array([False])

In [ ]:
test.duplicated().unique()

array([False])

# **Data preprocessing**

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [134]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

stop_words = stopwords.words('english')

def data_preprocessing(text):
  #Split into Words
  words = word_tokenize(text)

  #Lowercase
  words = [w.lower() for w in words]

  #Filter Out Punctuation
  words = [word for word in words if word.isalnum()]

  #Filter out Stop Words
  words = [w for w in words if not w in stop_words]
  
  return words

# **Features extraction**

In [135]:
first_questions = [data_preprocessing(f) for f in train["Question1"]]
second_questions = [data_preprocessing(s) for s in train["Question2"]]

**Sentence length**

In [149]:
question1_len = []
question2_len = []

for i in range(len(first_questions)):
  question1_len.append(len(first_questions[i]))
  question2_len.append(len(second_questions[i]))
print(len(question1_len))

283004


**The number of the common words and the number of the unique words in the questions**

In [ ]:
def compute_intersection(set1, set2):
  set1, set2 = set(set1), set(set2)
  return set1.intersection(set2)

def compute_union(set1, set2):
  set1, set2 = set(set1), set(set2)
  return set1.union(set2)

In [148]:
common_words = []
unique_words = []

for i in range(len(first_questions)):

  intersection = compute_intersection(first_questions[i], second_questions[i])
  union = compute_union(first_questions[i], second_questions[i])
  common_words.append(len(intersection))
  unique_words.append(len(union))

print(len(common_words))

283004


In [ ]:
set1 = set(first_questions[0])
set2 = set(second_questions[0])
intersection = set1.intersection(set2)
print(intersection)

{'market', 'guide', 'invest', 'step', 'share'}


**Jaccard Similarity**

In [ ]:
def jaccard_similarity(set1, set2): 
    # Find the intersection
    intersection = compute_intersection(set1, set2)
    # Find the union
    union = compute_union(set1, set2)
    # Calculate Jaccard similarity score 
    # using length of intersection set divided by length of union set
    return float(len(intersection)) / len(union)

In [140]:
jaccard_scores = []

for i in range(len(first_questions)):
  
  if(len(first_questions[i]) == 0 or len(second_questions[i]) == 0 ):
    jaccard_score = 0.0
  else:
    jaccard_score = jaccard_similarity(first_questions[i], second_questions[i])
  jaccard_scores.append(jaccard_score)
print(len(jaccard_scores))

common_words = []
unique_words = []

283004


**Cosine Similarity**

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
def cosine_similarity_score(s1, s2):
  # form a set containing keywords of both strings 
  X_set, Y_set = set(s1), set(s2)
  l1, l2 = [], []
  rvector = X_set.union(Y_set)  
  
  for w in rvector: 
      if w in X_set: l1.append(1) # create a vector 
      else: l1.append(0) 
      if w in Y_set: l2.append(1) 
      else: l2.append(0) 
  c = 0
    
  # cosine formula  
  for i in range(len(rvector)): 
          c+= l1[i]*l2[i] 
  cosine = c / float((sum(l1)*sum(l2))**0.5) 
  return cosine

In [152]:
cosine_similarity_score(first_questions[0], second_questions[0])

0.9128709291752769

In [151]:
cosine_scores = []

for i in range(len(first_questions)):

  if(len(first_questions[i]) == 0 or len(second_questions[i]) == 0 ):
    cosine_score = 0.0
  else:
    cosine_score = cosine_similarity_score(first_questions[i], second_questions[i])
  cosine_scores.append(cosine_score)
print(len(cosine_scores))

283004


**Building new dataframe**

In [153]:
new_train = pd.DataFrame({

    "Id": train["Id"], 
    "Length1": question1_len,
    "Length2": question2_len,
    "CommonWords": common_words,
    "UniqueWords": unique_words,
    "JaccardSimilarity": jaccard_scores,
    "CosineSimilarity": cosine_scores,
    })

new_train

,Id,Length1,Length2,CommonWords,UniqueWords,JaccardSimilarity,CosineSimilarity
0,0,7,6,5,6,0.833333,0.912871
1,1,3,8,2,9,0.222222,0.408248
2,2,6,5,2,9,0.222222,0.365148
3,3,3,5,0,8,0.000000,0.000000
4,4,10,5,2,13,0.153846,0.282843
...,...,...,...,...,...,...,...
282999,282999,4,17,3,17,0.176471,0.375000
283000,283000,2,2,2,2,1.000000,1.000000
283001,283001,6,6,5,7,0.714286,0.833333
283002,283002,5,5,2,7,0.285714,0.447214


**Model training**

In [154]:
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from sklearn.svm import SVC, LinearSVC
from sklearn.neighbors import KNeighborsClassifier

from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, recall_score, roc_auc_score, precision_score

from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

X_data = new_train.drop(['Id'], axis = 1)
y = train['IsDuplicate']
X_data

,Length1,Length2,CommonWords,UniqueWords,JaccardSimilarity,CosineSimilarity
0,7,6,5,6,0.833333,0.912871
1,3,8,2,9,0.222222,0.408248
2,6,5,2,9,0.222222,0.365148
3,3,5,0,8,0.000000,0.000000
4,10,5,2,13,0.153846,0.282843
...,...,...,...,...,...,...
282999,4,17,3,17,0.176471,0.375000
283000,2,2,2,2,1.000000,1.000000
283001,6,6,5,7,0.714286,0.833333
283002,5,5,2,7,0.285714,0.447214


In [155]:
#without cosine
X_train, X_test, y_train, y_test = train_test_split(X_data, y, test_size=0.2)
clf = RandomForestClassifier()
clf.fit(X_train, y_train)
preds = clf.predict(X_test)
print(classification_report(y_test, preds))

              precision    recall  f1-score   support

           0       0.77      0.74      0.76     35604
           1       0.59      0.63      0.61     20997

    accuracy                           0.70     56601
   macro avg       0.68      0.68      0.68     56601
weighted avg       0.70      0.70      0.70     56601



In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_data, y, test_size=0.2)
clf = RandomForestClassifier()
clf.fit(X_train, y_train)
preds = clf.predict(X_test)
print(classification_report(y_test, preds))

              precision    recall  f1-score   support

           0       0.77      0.74      0.76     35577
           1       0.59      0.62      0.61     21024

    accuracy                           0.70     56601
   macro avg       0.68      0.68      0.68     56601
weighted avg       0.70      0.70      0.70     56601



In [105]:
X_train, X_test, y_train, y_test = train_test_split(X_data, y, test_size=0.2)
clf = LinearSVC()
clf.fit(X_train, y_train)
preds = clf.predict(X_test)
print(classification_report(y_test, preds))

              precision    recall  f1-score   support

           0       0.74      0.72      0.73     35631
           1       0.54      0.57      0.55     20970

    accuracy                           0.66     56601
   macro avg       0.64      0.64      0.64     56601
weighted avg       0.67      0.66      0.66     56601



/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


**Cross-validation**

In [177]:
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, recall_score, roc_auc_score, precision_score

kf = KFold(n_splits=5)

X = X_data
kf.get_n_splits(X_data)
print(kf)

y = train["IsDuplicate"]

accuracy = 0
F1_score = 0
precision = 0
recall = 0

X_np = X.to_numpy()
y_np = y.to_numpy()

for train_index, test_index in kf.split(X_np):

  # print("TRAIN:", train_index, "TEST:", test_index)
  X_train, X_test = X_np[train_index], X_np[test_index]
  y_train, y_test = y_np[train_index], y_np[test_index]

  print(type(X_train))
  print(type(y_train))
  
  model = RandomForestClassifier()
  model.fit(X_train, y_train)
  y_pred = model.predict(X_test)

  accuracy += accuracy_score(y_test, y_pred)
  F1_score += f1_score(y_test, y_pred, average='macro') 
  precision += precision_score(y_test, y_pred, average='macro')
  recall += recall_score(y_test, y_pred, average='macro')

K = 5
print(accuracy/K, F1_score/K, precision/K, recall/K)

KFold(n_splits=5, random_state=None, shuffle=False)
(283004, 6)
(283004, 6)
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.series.Series'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
0.7011455770331568 0.6832772756970674 0.681789894591493 0.6854127584978073


**Working with Test Data**

In [106]:
Xt = test[['Id', 'Question1', 'Question2']]

for col in ['Question1', 'Question2']:
    Xt[col] = Xt[col].apply(data_preprocessing)

In [164]:
first_questions_test = [data_preprocessing(f) for f in test["Question1"]]
second_questions_test = [data_preprocessing(s) for s in test["Question2"]]

In [165]:
print(len(first_questions_test))

121287


In [167]:
question1_len_test = []
question2_len_test = []
common_words_test = []
unique_words_test = []
cosine_scores_test = []
jaccard_scores_test = []

for i in range(len(first_questions_test)):

  question1_len_test.append(len(first_questions_test[i]))
  question2_len_test.append(len(second_questions_test[i]))
  intersection = compute_intersection(first_questions_test[i], second_questions_test[i])
  union = compute_union(first_questions_test[i], second_questions_test[i])
  common_words_test.append(len(intersection))
  unique_words_test.append(len(union))

  if(len(first_questions_test[i]) == 0 or len(second_questions_test[i]) == 0 ):
    jaccard_score = 0.0
    cosine_score = 0.0
  else:
    jaccard_score = jaccard_similarity(first_questions_test[i], second_questions_test[i])
    cosine_score = cosine_similarity_score(first_questions_test[i], second_questions_test[i])
  jaccard_scores_test.append(jaccard_score)
  cosine_scores_test.append(cosine_score)

In [168]:
new_test = pd.DataFrame({
    "Length1": question1_len_test,
    "Length2": question2_len_test,
    "CommonWords": common_words_test,
    "UniqueWords": unique_words_test,
    "JaccardSimilarity": jaccard_scores_test,
    "CosineSimilarity": cosine_scores_test
    })

new_test

,Length1,Length2,CommonWords,UniqueWords,JaccardSimilarity,CosineSimilarity
0,8,7,3,12,0.250000,0.400892
1,5,5,3,7,0.428571,0.600000
2,5,4,4,5,0.800000,0.894427
3,10,10,9,11,0.818182,0.900000
4,16,5,1,20,0.050000,0.111803
...,...,...,...,...,...,...
121282,7,7,6,8,0.750000,0.857143
121283,3,3,2,4,0.500000,0.666667
121284,2,1,1,2,0.500000,0.707107
121285,9,13,0,22,0.000000,0.000000


In [169]:
clf = RandomForestClassifier()
clf.fit(X_data, y)
final_predictions = clf.predict(new_test)

In [170]:
final_output = pd.DataFrame({

    "Id": test["Id"], 
    "Predicted": final_predictions})

final_output

,Id,Predicted
0,283003,0
1,283004,1
2,283005,1
3,283006,0
4,283007,0
...,...,...
121282,404285,0
121283,404286,1
121284,404287,0
121285,404288,0


In [171]:
final_output.to_csv('drive/My Drive/Fall-2020/Big Data/Question2/duplicate_predictions.csv', sep=',', index=False)

In [173]:
output = pd.read_csv('drive/My Drive/Fall-2020/Big Data/Question2/duplicate_predictions.csv')
output

,Id,Predicted
0,283003,0
1,283004,1
2,283005,1
3,283006,0
4,283007,0
...,...,...
121282,404285,0
121283,404286,1
121284,404287,0
121285,404288,0


# **Combining with the Semantic Similarity Approach**

In [ ]:
import nltk
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

In [ ]:
from nltk import word_tokenize, pos_tag
from nltk.corpus import wordnet as wn

def penn_to_wn(tag):
    """ Convert between a Penn Treebank tag to a simplified Wordnet tag """
    if tag.startswith('N'):
        return 'n'
    if tag.startswith('V'):
        return 'v'
    if tag.startswith('J'):
        return 'a'
    if tag.startswith('R'):
        return 'r'
    return None
 
def tagged_to_synset(word, tag):
    wn_tag = penn_to_wn(tag)
    if wn_tag is None:
        return None
    try:
        return wn.synsets(word, wn_tag)[0]
    except:
        return None
 
def sentence_similarity(sentence1, sentence2):
    """ compute the sentence similarity using Wordnet """
    # Tag sentences
    sentence1 = pos_tag(sentence1)
    sentence2 = pos_tag(sentence2)

    # Get the synsets for the tagged words
    synsets1 = [tagged_to_synset(*tagged_word) for tagged_word in sentence1]
    synsets2 = [tagged_to_synset(*tagged_word) for tagged_word in sentence2]
 
    # Filter out the Nones
    synsets1 = [ss for ss in synsets1 if ss]
    synsets2 = [ss for ss in synsets2 if ss]
 
    score, count = 0.0, 0

    # For each word in the first sentence
    for synset in synsets1:
        sim_scores = []
        best_score = 0.0
        # Get the similarity value of the most similar word in the other sentence
        for ss in synsets2:
          sim_score = synset.wup_similarity(ss)
      
          if sim_score is not None:
            sim_scores.append(sim_score)

        if(len(sim_scores) != 0):
          best_score = max(sim_scores)
          # Check that the similarity could have been computed
          # if best_score is not None:
        score += best_score
        count += 1
 
    # Average the values

    if(score == 0.0):
      return 0.0
    score /= count
    return score
 
def symmetric_sentence_similarity(sentence1, sentence2):
    """ compute the symmetric sentence similarity using Wordnet """
    return (sentence_similarity(sentence1, sentence2) + sentence_similarity(sentence2, sentence1)) / 2 

In [ ]:
similarity_scores = []

for i in range(len(first_questions)):

  similarity_score = symmetric_sentence_similarity(first_questions[i], second_questions[i])
  similarity_scores.append(similarity_score)

print(len(similarity_scores))

**Training the model**

In [ ]:
X_data = new_train.drop(['Id'], axis = 1)
y = train['IsDuplicate']
X_data

In [178]:
X_data["SemanticSimilarity"] = np.array(similarity_scores)
X_data

,Length1,Length2,CommonWords,UniqueWords,JaccardSimilarity,CosineSimilarity,SemanticSimilarity
0,7,6,5,6,0.833333,0.912871,0.912500
1,3,8,2,9,0.222222,0.408248,0.483796
2,6,5,2,9,0.222222,0.365148,0.556962
3,3,5,0,8,0.000000,0.000000,0.204167
4,10,5,2,13,0.153846,0.282843,0.538204
...,...,...,...,...,...,...,...
282999,4,17,3,17,0.176471,0.375000,0.613756
283000,2,2,2,2,1.000000,1.000000,1.000000
283001,6,6,5,7,0.714286,0.833333,1.000000
283002,5,5,2,7,0.285714,0.447214,0.676007


In [ ]:
#without cosine
X_train, X_test, y_train, y_test = train_test_split(X_data, y, test_size=0.2)
clf = RandomForestClassifier()
clf.fit(X_train, y_train)
preds = clf.predict(X_test)
print(classification_report(y_test, preds))

              precision    recall  f1-score   support

           0       0.77      0.73      0.75     35637
           1       0.58      0.62      0.60     20964

    accuracy                           0.69     56601
   macro avg       0.67      0.68      0.67     56601
weighted avg       0.69      0.69      0.69     56601



In [179]:
X_train, X_test, y_train, y_test = train_test_split(X_data, y, test_size=0.2)
clf = RandomForestClassifier()
clf.fit(X_train, y_train)
preds = clf.predict(X_test)
print(classification_report(y_test, preds))

              precision    recall  f1-score   support

           0       0.77      0.73      0.75     35811
           1       0.57      0.62      0.60     20790

    accuracy                           0.69     56601
   macro avg       0.67      0.68      0.67     56601
weighted avg       0.70      0.69      0.69     56601



In [187]:
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, recall_score, roc_auc_score, precision_score

kf = KFold(n_splits=5)

X = X_data
kf.get_n_splits(X_data)
print(kf)


y = train["IsDuplicate"]

accuracy = 0
F1_score = 0
precision = 0
recall = 0

X_np = X.to_numpy()
y_np = y.to_numpy()

for train_index, test_index in kf.split(X_np):

  # print("TRAIN:", train_index, "TEST:", test_index)
  X_train, X_test = X_np[train_index], X_np[test_index]
  y_train, y_test = y_np[train_index], y_np[test_index]

  print(type(X_train))
  print(type(y_train))
  
  model = RandomForestClassifier()
  model.fit(X_train, y_train)
  y_pred = model.predict(X_test)

  accuracy += accuracy_score(y_test, y_pred)
  F1_score += f1_score(y_test, y_pred, average='macro') 
  precision += precision_score(y_test, y_pred, average='macro')
  recall += recall_score(y_test, y_pred, average='macro')

K = 5
print(accuracy/K, F1_score/K, precision/K, recall/K)

KFold(n_splits=5, random_state=None, shuffle=False)
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
0.6893047506995688 0.6716510168749406 0.6700037449183757 0.6742673949295934


**Working with the test set**

In [ ]:
similarity_scores_test = []

for i in range(len(first_questions_test)):

  similarity_score = symmetric_sentence_similarity(first_questions_test[i], second_questions_test[i])
  similarity_scores_test.append(similarity_score)
  
print(len(similarity_scores_test))

In [182]:
new_test2 = pd.DataFrame({
    "Length1": question1_len_test,
    "Length2": question2_len_test,
    "CommonWords": common_words_test,
    "UniqueWords": unique_words_test,
    "JaccardSimilarity": jaccard_scores_test,
    "CosineSimilarity": cosine_scores_test,
    "SemanticSimilarity": similarity_scores_test
    })

new_test2

,Length1,Length2,CommonWords,UniqueWords,JaccardSimilarity,CosineSimilarity,SemanticSimilarity
0,8,7,3,12,0.250000,0.400892,0.584314
1,5,5,3,7,0.428571,0.600000,0.708333
2,5,4,4,5,0.800000,0.894427,0.910714
3,10,10,9,11,0.818182,0.900000,0.881061
4,16,5,1,20,0.050000,0.111803,0.219322
...,...,...,...,...,...,...,...
121282,7,7,6,8,0.750000,0.857143,0.944444
121283,3,3,2,4,0.500000,0.666667,0.833333
121284,2,1,1,2,0.500000,0.707107,1.000000
121285,9,13,0,22,0.000000,0.000000,0.337253


In [184]:
clf = RandomForestClassifier()
clf.fit(X_data, y)
final_predictions = clf.predict(new_test2)

In [ ]:
final_output2 = pd.DataFrame({

    "Id": test["Id"], 
    "Predicted": final_predictions})

final_output2

In [185]:
final_output.to_csv('drive/My Drive/Fall-2020/Big Data/Question2/TrialSem/duplicate_predictions.csv', sep=',', index=False)
output = pd.read_csv('drive/My Drive/Fall-2020/Big Data/Question2/TrialSem/duplicate_predictions.csv')
output

,Id,Predicted
0,283003,0
1,283004,1
2,283005,1
3,283006,0
4,283007,0
...,...,...
121282,404285,0
121283,404286,1
121284,404287,0
121285,404288,0
